In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA

# 
from tqdm import tqdm
import sleap

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage

#
import parmap

import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.database import database

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
cohort = database.CohortProcessor()
cohort.fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'
cohort.load_database()


DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename
0         0.0     2.0     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_09_53_59_813912_compressed_Day.slp
1         1.0     2.0     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_16_08_629297_compressed_Day.slp
2         2.0     2.0     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_38_20_043555_compressed_Day.slp
3         3.0     2.0     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_11_00_31_566455_compressed_Day.slp
4         4.0     2.0     Day  2020_07_20_11_22_36_823934_compressed.mp4   

In [21]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################
cohort.n_cores = 30
cohort.parallel = True
cohort.preprocess_feature_tracks()



100%|██████████| 126/126 [00:00<00:00, 3307.29it/s]


In [24]:
#################################################################
###################### PREPROCESS HUDDLES #######################
#################################################################

# 
cohort.parallel = True
cohort.preprocess_huddle_tracks()



100%|██████████| 126/126 [00:15<00:00,  7.99it/s]


In [ ]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################
cohort.n_cores = 30



In [67]:
#
res=[]
for k in range(0,991,1):
    cohort.track_id = k
    track = cohort.load_single_feature_spines()

    #
    if track is None:
        res.append(np.zeros((6,6)))
        continue
        
    #
    cohort.symmetric_matrices=False
    cohort.plotting=False
    res.append(cohort.compute_pairwise_interactions(track))
    
res = np.array(res)
print ("res: ", res.shape)

res:  (991, 6, 6)


In [78]:
names=['female','male','pup1','pup2','pup3','pup4']
from itertools import combinations

pairs = list(combinations(np.arange(6),2))

#
Pstart = 15
plt.figure()
t = np.arange(res.shape[0])/3./24+Pstart
pairs=[[0,1]]
for pair in pairs:
    plt.plot(t,res[:,pair[0],pair[1]], label = str(names[pair[0]])+ " and " +str(names[pair[1]]))

plt.xlabel("Time (~days)")
plt.title("Time spent together")
plt.legend()
#plt.label("% of total time")
plt.ylim(bottom=0)
plt.xlim(t[0],t[-1])

plt.show()